In [ ]:
#Importing Required Libraries
import pandas as pd
import json
import requests
import time
import random

In [ ]:
#Uploading Jstor Data as a Data Frame
file_path = "data/jstor_data.jsonl"
jstor_df = pd.read_json(file_path, lines=True)

In [ ]:
#Inspecting Jstor Data Frame
jstor_df.value_counts("docType")

In [ ]:
#Inspecting Jstor Data Frame
jstor_df.columns

In [ ]:
#Droping NA values for DOI
jstor_df = jstor_df.dropna(subset=['doi'])

In [ ]:
#Retrieving DOIs for Crossref API Call
dois_jstor = jstor_df["doi"].reset_index()

In [ ]:
#Setting Header for Crossref API call
header = {'User-Agent': 'Shivam Sen(Collecting Data for Masters Thesis; mailto:shivamsen910@gmail.com; Institution: Charles University, Prague)'}

In [ ]:
#Setting URL for Crossref API call
url = "https://api.crossref.org/works/"+dois_jstor.loc[0,'doi']

In [ ]:
#Making First Crossref API Call
r = requests.get(url, headers=header)

In [ ]:
#Processing Response to Data Frame
message = json.loads(r.content)['message']
temp_keys = list(message.keys())
temp_values = list(message.values())
meta_jstor = pd.DataFrame([temp_values], columns=temp_keys)

In [ ]:
#Making Remaining API Calls for Jstor batch Metadata
for i in range(1,len(dois_jstor)):
    url = "https://api.crossref.org/works/"+dois_jstor.loc[i,'doi']
    r = requests.get(url, headers=header)
    message = json.loads(r.content)['message']
    temp_keys = list(message.keys())
    temp_values = list(message.values())
    temp_df = pd.DataFrame([temp_values], columns=temp_keys)
    meta_jstor = pd.concat([meta_jstor, temp_df], axis=0, join='outer')
    time.sleep(3 + random.randint(3, 5))

In [ ]:
#Saving Jstor Metadata into CSV Files
meta_jstor.to_csv("data/jstor_metadata.csv", index=False)